## Run ZimT simulations and plot the results
Version 0.0.1
(c) Vincent M. Le Corre (https://github.com/VMLC-PV)

In [ ]:
# Package import
%matplotlib inline
import os,platform,warnings,itertools
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

# Import homemade package by VLC
from SIMsalabim_utils.tVG_gen import *
from SIMsalabim_utils.RunSim import *
from SIMsalabim_utils.SimSS_plots import *
from SIMsalabim_utils.aux_func import *
from SIMsalabim_utils.plot_settings_screen import *

## General Inputs
warnings.filterwarnings("ignore")           # Don't show warnings
system = platform.system()                  # Operating system
max_jobs = os.cpu_count()-2                 # Max number of parallel simulations (for number of CPU use: os.cpu_count() )
do_multiprocessing = False                  # Use multiprocessing
if system == 'Windows':                     # cannot easily do multiprocessing in Windows
        max_jobs = 1
        do_multiprocessing = False
        try:                                # kill all running jobs to avoid conflicts
            os.system('taskkill.exe /F /IM simss.exe')
        except:
            pass

# Path to SIMsalabim
path2simu = os.path.join(os.getcwd() , 'Simulation_program','SIMsalabimv456','ZimT')

In [ ]:
## Simulation Inputs
Vstart = 1.17                                               # Start voltage (unit: V)
Vfinal = 0.0                                                  # Final voltage (unit: V)
scans = np.geomspace(1e-2, 1e4,num=10)                      # Scan speed (unit: V/s)
Gens = [3.4e+27]                                           # Average generation rate (unit: m^-3 s^-1) 
steps = 500                                                 # Number of voltage step
fixed_str = ''                                              # add any fixed string to the simulation command     
run_simu = True                                             # Run simulation

In [ ]:
# Prepare the Simulation Inputs and tVG files
str_lst,output_files,Var_files,scPars_files,code_name_lst,path_lst,labels,tVG_lst = [],[],[],[],[],[],[],[]

for scan in scans:
    for Gen in Gens:
        zimt_JV_double_sweep(Vstart,Vfinal,scan,Gen,steps,tVG_name=os.path.join(path2simu,'tVG_JV_Hyst_scan_{:.2e}_G_{:.2e}.txt'.format(scan,Gen))) ## Prepare tVG files (Can be any of the experiment described in tVG_gen.py)
        str_lst.append(fixed_str+' -tVG_file tVG_JV_Hyst_scan_{:.2e}_G_{:.2e}.txt -tj_file tj_JV_Hyst_scan_{:.2e}_G_{:.2e}.dat'.format(scan,Gen,scan,Gen))
        code_name_lst.append('zimt')
        path_lst.append(path2simu)
        tVG_lst.append(os.path.join(path2simu,'tVG_JV_Hyst_scan_{:.2e}_G_{:.2e}.txt'.format(scan,Gen)))
        output_files.append(os.path.join(path2simu,'tj_JV_Hyst_scan_{:.2e}_G_{:.2e}.dat'.format(scan,Gen)))
        labels.append(str('{:.2e} '.format(scan) + 'V s$^{-1}$ G$_{ehp}$' +' {:.2e} '.format(Gen)+'m$^{-3}$ s$^{-1}$'))
        Var_files.append('none')

Simulation_Inputs = str_lst,output_files,Var_files,scPars_files,code_name_lst,path_lst,labels # Prepare the Simulation Inputs

# Run the simulation
if run_simu:
    RunSimulation(Simulation_Inputs,max_jobs=max_jobs,do_multiprocessing=do_multiprocessing,verbose=True) # runs the simulations


In [ ]:
# Plot the simulation results
colors = plt.cm.viridis(np.linspace(0,1,max(len(str_lst),3)+1)) # prepare color for plots
idx = 0
Voc_dir1,Jsc_dir1,FF_dir1,PCE_dir1 = [],[],[],[]
Voc_dir2,Jsc_dir2,FF_dir2,PCE_dir2 = [],[],[],[]
plt.figure(0,figsize=(16,12)) 
for scan in scans:
    for Gen in Gens:
            data_tj = pd.read_csv(os.path.join(path2simu,'tj_JV_Hyst_scan_{:.2e}_G_{:.2e}.dat'.format(scan,Gen)),delim_whitespace=True)
            
            # Seperate scan directions
            # abs(data_Var.Vext -Vext) == min(abs(data_Var.Vext -Vext))
            switch = data_tj.index[abs(data_tj['Vext']-Vfinal)==min(abs(data_tj['Vext']-Vfinal))].tolist() # get index switch scan direction
            scan_dir1 = data_tj.head(switch[0]+1)
            scan_dir2 = data_tj.tail(switch[0]+1)
            if Vstart > Vfinal: # reorder dataframe so that the perf can be calculated
                scan_dir1 = scan_dir1[::-1].reset_index()
            else:
                scan_dir2 = scan_dir2[::-1].reset_index()

            # Get perfs
            Jsc_dir1.append(get_Jsc(scan_dir1['Vext'],scan_dir1['Jext'])/10)
            Voc_dir1.append(get_Voc(scan_dir1['Vext'],scan_dir1['Jext']))
            FF_dir1.append(get_FF(scan_dir1['Vext'],scan_dir1['Jext']))
            PCE_dir1.append(get_PCE(scan_dir1['Vext'],scan_dir1['Jext'])/10)
            Jsc_dir2.append(get_Jsc(scan_dir2['Vext'],scan_dir2['Jext'])/10)
            Voc_dir2.append(get_Voc(scan_dir2['Vext'],scan_dir2['Jext']))
            FF_dir2.append(get_FF(scan_dir2['Vext'],scan_dir2['Jext']))
            PCE_dir2.append(get_PCE(scan_dir2['Vext'],scan_dir2['Jext'])/10)
            
            plt.plot(scan_dir1['Vext'],scan_dir1['Jext']/10,color=colors[idx],linestyle='-',marker='None')
            plt.plot(scan_dir2['Vext'],scan_dir2['Jext']/10,color=colors[idx],linestyle='--',marker='None')
            
            idx = idx + 1 
plt.xlabel('Applied Voltage [V]')
plt.ylabel('Current Density [mA cm$^{-2}$]')
plt.xlim([0,1.2])
plt.ylim([-25,1e1])
plt.grid(visible=True,which='both')

# Get experimental data
data_exp = pd.read_csv(os.path.join(path2simu,'pero_exp_data.dat'),delim_whitespace=True)

# Plot the experimental data
plt.figure(1,figsize=(16,12)) 
plt.subplot(221)
plt.semilogx(scans,PCE_dir2,color='tab:blue',linestyle='-',marker='None',markersize=10,markerfacecolor='w',label='Forward')
plt.semilogx(scans,PCE_dir1,color='tab:orange',linestyle='-',marker='None',markersize=10,markerfacecolor='w',label='Reverse')
PCE_for = data_exp['FF_for_mean']*data_exp['Voc_for_mean']*abs(data_exp['Jsc_for_mean'])
PCE_rev = data_exp['FF_rev_mean']*data_exp['Voc_rev_mean']*abs(data_exp['Jsc_rev_mean'])
plt.semilogx(data_exp['speed_mean'],PCE_for,linestyle='None',marker='o',markersize=10,markerfacecolor='tab:blue')
plt.semilogx(data_exp['speed_mean'],PCE_rev,linestyle='None',marker='o',markersize=10,markerfacecolor='tab:orange')

plt.xlabel('Scan speed [V s$^{-1}$]')
plt.ylabel('PCE [%]')
plt.ylim([10,25])
plt.legend(loc='best', title="Scan direction:")
plt.grid(visible=True,which='both')
plt.subplot(222)
plt.semilogx(scans,FF_dir2,linestyle='-',marker='None',markersize=10,markerfacecolor='w')
plt.semilogx(scans,FF_dir1,linestyle='-',marker='None',markersize=10,markerfacecolor='w')
plt.semilogx(data_exp['speed_mean'],data_exp['FF_for_mean'],linestyle='None',marker='o',markersize=10,markerfacecolor='tab:blue',markeredgecolor='tab:blue')
plt.semilogx(data_exp['speed_mean'],data_exp['FF_rev_mean'],linestyle='None',marker='o',markersize=10,markerfacecolor='tab:orange',markeredgecolor='tab:orange')
plt.xlabel('Scan speed [V s$^{-1}$]')
plt.ylabel('FF')
plt.ylim([0.5,1])
plt.grid(visible=True,which='both')
plt.subplot(223)
plt.semilogx(scans,Voc_dir2,linestyle='-',marker='None',markersize=10,markerfacecolor='w')
plt.semilogx(scans,Voc_dir1,linestyle='-',marker='None',markersize=10,markerfacecolor='w')
plt.semilogx(data_exp['speed_mean'],data_exp['Voc_for_mean'],linestyle='None',marker='o',markersize=10,markerfacecolor='tab:blue',markeredgecolor='tab:blue')
plt.semilogx(data_exp['speed_mean'],data_exp['Voc_rev_mean'],linestyle='None',marker='o',markersize=10,markerfacecolor='tab:orange',markeredgecolor='tab:orange')
plt.xlabel('Scan speed [V s$^{-1}$]')
plt.ylabel('V$_{OC}$ [V]')
plt.ylim([0.9,1.3])
plt.grid(visible=True,which='both')
plt.subplot(224)
plt.semilogx(scans,abs(np.asarray(Jsc_dir2)),linestyle='-',marker='None',markersize=10,markerfacecolor='w')
plt.semilogx(scans,abs(np.asarray(Jsc_dir1)),linestyle='-',marker='None',markersize=10,markerfacecolor='w')
plt.semilogx(data_exp['speed_mean'],abs(data_exp['Jsc_for_mean']),linestyle='None',marker='o',markersize=10,markerfacecolor='tab:blue',markeredgecolor='tab:blue')
plt.semilogx(data_exp['speed_mean'],abs(data_exp['Jsc_rev_mean']),linestyle='None',marker='o',markersize=10,markerfacecolor='tab:orange',markeredgecolor='tab:orange')
plt.xlabel('Scan speed [V s$^{-1}$]')
plt.ylabel('J$_{SC}$ [mA cm$^{-2}$]')
plt.ylim([18,26])

plt.tight_layout()
plt.grid(visible=True,which='both')

In [ ]:
# Clean output files from simulation folders
from SIMsalabim_utils.CleanFolder import *

Do_Cleaning = False # Careful, this will delete all files in the folder
if Do_Cleaning:
    clean_up_output('tj',path2simu)
    clean_up_output('tVG',path2simu)
    clean_up_output('JV',path2simu)
    clean_up_output('Var',path2simu)
    clean_up_output('scPar',path2simu)
    clean_up_output('Str4Parallel',path2simu)